In [1]:

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import netCDF4
import plotly.graph_objects as go
import plotly.express as pex
import plotly.figure_factory as ff
import plotly.io as pio
from IPython.display import display
import pickle as pk
import geopandas
import geoplot
from shapely import wkt

In [2]:
#initializing data
ny_counties = pd.read_csv("/Users/nicholasbattin/Downloads/SLR Data Science Folder/slr-data-surfers-main/ELEVATION_20240402.csv")
ny_counties["the_geom"] = ny_counties["the_geom"].apply(wkt.loads)
ny_counties = geopandas.GeoDataFrame(ny_counties, geometry="the_geom")

In [ ]:
ny_counties.head()

In [3]:
ny_counties.drop(["FEAT_CODE", "SOURCE_ID", "SUB_CODE", "STATUS"], axis=1, inplace=True)

In [4]:
ny_counties['lon'] = ny_counties.the_geom.x
ny_counties['lat'] = ny_counties.the_geom.y
ny_counties.head()
ny_counties.to_csv("modified_ny_counties.csv")

In [7]:
def elevationMap(counties):
    ax = plt.axes(projection="3d")
    x = counties["lon"].to_numpy()
    y = counties["lat"].to_numpy()
    X, Y = np.meshgrid(x, y)
    ax.plot(X, Y)
    plt.show()
    return ax

In [8]:
elevationMap(ny_counties)

: 

In [ ]:
def projectSurface(surface, counties):
    z = counties["ELEVATION"].to_numpy()
    

In [ ]:
filename = 'slr_all_us_11_21.pkl'

def pk_to_pd(path):
    with open(path, 'rb') as f:
        data = pk.load(f)
    time = data['time']
    latitude = data['latitude']
    longitude = data['longitude']
    adt = data['adt']

    # Since your dataset likely represents a 2D grid of points over multiple times, 
    # we'll need to repeat the time array to match the number of latitude and longitude points
    time_repeated = np.repeat(time, latitude.size * longitude.size)

    # Create a meshgrid for latitude and longitude, then flatten them for the DataFrame
    lon, lat = np.meshgrid(longitude, latitude)
    lon_flat = lon.flatten()
    lat_flat = lat.flatten()

    # Repeat latitude and longitude coordinates for each time step
    lat_repeated = np.tile(lat_flat, len(time))
    lon_repeated = np.tile(lon_flat, len(time))

    adt_flat = adt.flatten().data  # Or another method to match DataFrame's structure

    # Now combine everything into a DataFrame
    df = pd.DataFrame({
        'Time': time_repeated,
        'Latitude': lat_repeated,
        'Longitude': lon_repeated, 
        'adt': adt_flat,
    })
    return df

df = pk_to_pd(filename)

In [ ]:
def transform(df: pd.DataFrame, year: int, n_days=1)-> pd.DataFrame:
    df = df[(df['Latitude'] > 40.2) & (df['Latitude'] < 45)] #latitude range of nyc

    df = df[(df['Longitude'] > -79) & (df['Longitude'] < -71)] #longitude range of nyc
    
    df["Time"] = pd.to_datetime(df["Time"]) #converting time to datetime
    
    df = df[(df['adt'] > -2.147484e+07)] #values above minimum
    
    df = df.loc[df['Time'].dt.year == year] #comment out if doing whole data

    df['Rate_of_Change'] = df.groupby(['Latitude', 'Longitude'])['adt'].diff(periods=n_days) #rate of change in adt

    df = df.groupby(['Latitude', 'Longitude']).apply(lambda x: x.iloc[n_days-1::n_days]).reset_index(drop=True)
    df.fillna(0, inplace=True)

    return df

yearly_df = transform(df, 2015)
yearly_df.shape

In [ ]:
def ny_county_map(slr_df):
    ax = plt.axes(projection="3d")
    
map = ny_county_map(yearly_df)
map